In [1]:
import pandas as pd
import numpy as np
import slugify
import datetime as dt
from IPython.display import Markdown as md
%load_ext watermark


# this defines the css rules for the note-book table displays
header_row = {'selector': 'th:nth-child(1)', 'props': f'background-color: #FFF; text-align:right'}
even_rows = {"selector": 'tr:nth-child(even)', 'props': f'background-color: rgba(139, 69, 19, 0.08);'}
odd_rows = {'selector': 'tr:nth-child(odd)', 'props': 'background: #FFF;'}
table_font = {'selector': 'tr', 'props': 'font-size: 10px;'}
table_data = {'selector': 'td', 'props': 'padding: 6px;'}
table_caption = {'selector': 'caption', 'props': 'font-size: 14px; font-style: italic; caption-side: bottom; text-align: left; margin-top: 10px'}
table_large_data = {'selector': 'tr', 'props': 'font-size: 14px;'}


table_css_styles = [even_rows, odd_rows, table_font, header_row, table_caption]
table_large_font = [even_rows, odd_rows, table_large_data, header_row, table_caption]

# this picks up the correct display names for the different languages
display_names_fr = {'loc_date': 'échantillons',
 'city': 'municipalités',
 'quantity': 'quantité',
 'location': 'site',
 'length': 'mètres',
 'area': 'm²',
 'hours': 'heures',
 'count': 'echantillons',
 'mean': 'moyenne',
 'std': 'écart-type'}

# Les variables aléatoires

Il s'agit de l'ensemble de données de base que vous pouvez obtenir une fois que vous avez supprimé les colonnes indésirables et appliqué un regroupement catégoriel. Ici, nous utilisons les groupes limités par la géographie, le temps, l'objet et l'ampleur.

## Le contexte de l'observation

Une fois les données collectées et agrégées, d'autres informations sont nécessaires pour les placer dans leur contexte par rapport aux autres observations.

Dans ce modèle, les étiquettes sont largement utilisées. Les objets, les lieux et les attributs des objets et des lieux ont tous des étiquettes. Les étiquettes relatives à l'utilisation des sols correspondent au rapport entre l'attribut et l'espace disponible.



```{attention}

_À l'origine, cette méthode était destinée à la surveillance des déchets sur le littoral. L'application de cette méthode aux rivières et aux sentiers de montagne fait l'objet de la présente discussion._

Avoir un consensus sur les descriptions d'objets et être discipliné dans les données enregistrées : c'est la meilleure façon de capturer les informations de l'observateur.

* Une fois le consensus atteint, les ingénieurs peuvent se mettre au travail.

* Il est plus facile de chercher quelque chose si vous savez ce que vous cherchez ou si vous savez ce qui est important.
```

In [2]:
# lakes
wt_data = pd.read_csv("resources/u_wt_data.csv")
wt_data["date"] = pd.to_datetime(wt_data["date"], format="%Y-%m-%d")

# start variables
# this is the observations from the begining of monitoring
# to the end of the federal report from 2021. 
start_date, end_date = "2015-11-15", "2021-5-31"
lake = 'lac-leman'
catchment = 'rhone'
some_quants = [.03, .25, .48, .5, .52, .75, .97]

d_mask = (wt_data['date'] >= start_date) & (wt_data['date'] <= end_date)
wt_data = wt_data[d_mask].copy()

wt_data = wt_data[wt_data.water_name_slug == lake].copy()
wt_data.rename(columns={"pcs_m":"pcs/m"}, inplace=True)

# the start and end dates are a sample period. There is
# data after the end date. It is called testing
wt_data["Project"] = "training"

survey_data_columns = ["loc_date", "location",  "water_name_slug", "city" ,"date", "doy", "Project", "code", "pcs/m", "quantity"]
groupby_columns = ["loc_date", "location", "water_name_slug", "city", "date", "doy", "Project", "code"]
operators = {"pcs/m": 'sum', 'quantity':'sum'}

wt_data = wt_data[survey_data_columns].groupby(groupby_columns, as_index=False).agg(operators)

# saint sulpice after may 31 2021
sup = pd.read_csv("resources/sup_after.csv")

# collected by ASL 2022
nxc = pd.read_csv("resources/u_pstk.csv")

# these are locations that do not have land use data
not_these = ['amphion', 'anthy', 'excenevex', 'lugrin', 'meillerie', 'saint-disdille', 'tougues']

nxc = nxc[~nxc.location.isin(not_these)].copy()
not_in_here = ["vidy-pk", "versoix-pk", "tolochenaz-pk", "preverenges-pk"]
in_here = {x[:-3]:x[:-1] for x in not_in_here}
nxc_g = nxc[nxc.slug.isin(in_here.keys())].copy()
nxc_g["loc"] = nxc_g.slug.apply(lambda x: in_here[x])

nxc_gi = nxc[~nxc.slug.isin(in_here.keys())].copy()
nxc_gi["loc"] = nxc_gi.slug.where(nxc_gi.slug != "versoix", "versoix-p")
gtd = pd.concat([nxc_gi, nxc_g])
gtd = gtd.drop("location", axis=1)
gtd.rename(columns={"loc":"location"}, inplace=True)
gtd["water_name_slug"] = "lac-leman"


# all the data from le lac
# separated into to two groups: training and testing
combined = pd.concat([wt_data, sup[survey_data_columns], gtd[survey_data_columns]])
combined = combined.drop_duplicates(["loc_date", "code"])
combined["date"] = pd.to_datetime(combined["date"])
combined["date"] =combined["date"].dt.date


# codes and beaches
dfCodes = pd.read_csv("resources/codes.csv")
dfBeaches = pd.read_csv("resources/beaches.csv")
dfBeaches.set_index("slug", inplace=True)
dfBeaches = dfBeaches[dfBeaches.water_name_slug == lake].copy()

### Résultats d'inventaire

```{important}
C'est ce qui est compté. La manière dont ces objets sont classés est un élément essentiel des données. C'est aussi la façon la plus simple de faire mesurer une valeur. Il est plus facile et plus rapide de le faire au moment de l'enquête.
```
#### Début: nov 2015, fin: décembre 2022

In [3]:
caption = "L'inventaire de base, _jdl = jour de l'année_. Échantillons est l'identifiant de \
l'inventaire, il connecte les objets au même inventaire. La quantité \
et les pcs/m sont les variables numériques pour chaque ligne. Le \
reste des colonnes sont des étiquettes."

# add or edit the display names
display_names_fr.update({
    "doy":"jdl",
    "Project":"Projet"})

combined_d = combined.rename(columns=display_names_fr)

combined_d.head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3)

,échantillons,site,water_name_slug,municipalités,date,jdl,Projet,code,pcs/m,quantité
0,"('anarchy-beach', '2018-04-02')",anarchy-beach,lac-leman,La Tour-de-Peilz,2018-04-02,92,training,G1,0.000,0
1,"('anarchy-beach', '2018-04-02')",anarchy-beach,lac-leman,La Tour-de-Peilz,2018-04-02,92,training,G10,0.060,4
2,"('anarchy-beach', '2018-04-02')",anarchy-beach,lac-leman,La Tour-de-Peilz,2018-04-02,92,training,G100,0.140,10
3,"('anarchy-beach', '2018-04-02')",anarchy-beach,lac-leman,La Tour-de-Peilz,2018-04-02,92,training,G101,0.000,0
4,"('anarchy-beach', '2018-04-02')",anarchy-beach,lac-leman,La Tour-de-Peilz,2018-04-02,92,training,G102,0.000,0


### Détails du site d'inventaire

Celui-ci contient les détails de localisation tels que le GPS, le bassin fluvial de la ville, etc. Une grande partie a déjà été ajoutée aux données pour le travail. Cependant, les noms propres peuvent être extraits et des attributs peuvent être ajoutés à cet ensemble, puis appliqués.

La localisation GPS constitue le centre d'un hexagone de 3 000 mètres. La configuration de l'utilisation du sol pour chaque emplacement est configurée à partir de ce point.

```{note}
Les données utilisées pour identifier un emplacement sont définies sur la manière dont les informations vont être utilisées. C’est également un moyen simple d’améliorer le flux de données une fois les exigences comprises.
```

In [4]:
beach_columns = ['location', 'latitude', 'longitude',
       'water_name', 'city',
       'river_bassin']

display_names_fr.update({
    "water_name": "zone d'interêt",
    "river_bassin": "bassin versant",
})

caption = "Détails de localisation des plages du Léman. Des attributs \
peuvent être ajoutés à cet ensemble de données et transmis ultérieurement \
si nécessaire. Pour l'instant, les caractéristiques topographiques sont \
stockées sur un autre ensemble de données partageant le même index."
dfBeaches.index.name = None

    
dfBeaches[beach_columns].rename(columns=display_names_fr).head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3)

,site,latitude,longitude,zone d'interêt,municipalités,bassin versant
anarchy-beach,Anarchy Beach,46.447,6.860,Lac Léman,La Tour-de-Peilz,rhone
arabie,Arabie,46.462,6.835,Lac Léman,Vevey,rhone
baby-plage-geneva,Baby Plage Geneva,46.209,6.163,Lac Léman,Genève,rhone
baby-plage-ii-geneve,Baby-Plage II Genève,46.209,6.164,Lac Léman,Genève,rhone
bain-des-dames,Bain des Dames,46.451,6.858,Lac Léman,La Tour-de-Peilz,rhone


### Catégories d'objets

Les catégories d'objets correspondent à la manière dont la personne transmet l'observation à l'appareil. La description et le nom du groupe permettent aux administrations régionales d'obtenir davantage d'informations à partir des données.

```{note}
En créant des groupes d'objets qui représentent des cas d'utilisation, il peut être plus facile pour les acteurs régionaux d'identifier les priorités. Il s'agit peut-être de la meilleure voie de discussion entre les acteurs régionaux. Si les catégories d'objets ou les groupes correspondent à des priorités communes, les communications peuvent être simplifiées.
```

In [5]:
caption = "Tableau d'informations sur les objets. Nous pouvons ajouter des codes spécifiques à \
une région. Par exemple nous ajoutons le code G708 pour les bâtons de ski. \
Ils ne sont pas communs au bord des lacs mais on les trouve en montagne. \
Un autre exemple est celui des contenants d’appâts aux phéromones. \
Ils sont utilisés dans les vignobles et on les retrouve là où se concentrent les vignobles."

display_names_fr.update({
    "material": "matériel",
    "parent_code": "code de base",
    "groupname": "groupe"})


dfCodes.rename(columns=display_names_fr).head().style.set_table_styles(table_large_font).set_caption(caption)

,code,matériel,description,code de base,groupe
0,G708,Metal,Batons de ski,G199,recreation
1,G212,Chemicals,Oil nodules or coal fragments -- not from a BBQ,Parent code,unclassified
2,G213,Chemicals,Paraffin wax,Parent code,recreation
3,G214,Chemicals,Oil/tar,Parent code,infrastructure
4,G135,Cloth,"Clothes, footware, headware, gloves",Parent code,personal items


### Utilisation du sol

Il s’agit de l’ampleur des caractéristiques topographiques trouvées dans un rayon de 1 500 du lieu d’enquête. La source est la collection de cartes swissTLMRegio. Une superposition hexagonale est appliquée aux couches de couverture terrestre et d’utilisation des terres. Ensuite, le contenu de chaque hex est enregistré en mètres² ou en mètres linéaires.

Ce processus a montré des corrélations entre l'ampleur des résultats de l'enquête pour certains objets et l'ampleur de l'élément topographique. Ce processus a été expliqué dans [Near of Far](https://hammerdirt-analyst.github.io/landuse/probability.html) et  [Land use profile](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/land_use_correlation.html).

L'ampleur de l'élément topographique et sa valeur mise à l'échelle sont disponibles. Les données ont été séparées en groupes percentiles. Il existe 21 groupes, un groupe étant destiné au cas où la caractéristique d'utilisation des terres n'est pas présente, puis un groupe pour chaque cinquième percentile.

```{note}

Pour cette phase initiale, seules les couches cartographiques de base ont été utilisées. Pour les partenaires intéressés, l’une des couches cartographiques de[TLMRegio](https://www.swisstopo.admin.ch/de/geodata/landscape/tlmregio.html) fonctionnera sur ces données. Cela permet aux équipes indépendantes d’utiliser plus facilement les résultats pour des besoins spécifiques.

```

In [6]:
land_use = pd.read_csv("resources/u_land_use_cover_streets_rivers.csv")

display_names_fr.update({
    "slug":"site",
    "hex minus": "terre sèche",
    "use":"utilisation",
    "scaled":"normalisée",
    "binned":"rang"})

land_use_names_fr = {
    "See":"lac",
    "Wald":"forêt",
    "Siedl":"bâtiments",
    "dist": "distance à la rivière",
    "undefined":"non définie",
    "Obstanlage":"vergers",
    "Reben":"vignobles",
    "recreation":"récréation",
    "Strasse":"rues",
    "Stadtzentr":"centre ville",
    "Sumpf": "marécages",
    "Fels": "falaise",
    "Stausee":"réservoir",
    "Geroel":"rochers",
    "infrastructure":"infrastructure",
}

caption = "Le profil d'occupation du sol de la tiger-duck-beach à \
Saint Sulpice. Rappelez-vous que la grandeur des rues est en mètres, \
tout le reste est en mètres carrés. La quantité de terre sèche est \
donnée en mètres², normalisée étant le résultat de la magnitude/terre sèche. \
Rang est le classement de cette caractéristique d'utilisation des sols (0 - 20)."


# display land use for one location
lud = land_use[land_use.slug == "tiger-duck-beach"].rename(columns=display_names_fr)
lud = lud[~lud.utilisation.isin(["See", "dist"])].copy()
lud["rang"] = lud.rang.astype(int)
lud["magnitude"] = lud.magnitude.astype(int)
lud["utilisation"] = lud["utilisation"].apply(lambda x: land_use_names_fr[x])
lud["normalisée"] = lud["normalisée"].round(3)
lud.set_index("utilisation", inplace=True)
lud.drop("site", axis=1, inplace=True)
lud.index.name = None
lud.style.set_table_styles(table_large_font).set_caption(caption).format('{:,}')

,magnitude,terre sèche,normalisée,rang
bâtiments,"3,351,943","3,689,451",0.909,19
forêt,"157,389","3,689,451",0.043,1
non définie,"180,119","3,689,451",0.049,1
infrastructure,"1,391,466","3,689,451",0.377,8
récréation,"249,970","3,689,451",0.068,2
rues,"95,764",0,0.485,10


In [7]:
two = ['aubonne-pk', 'baby-plage-pk', 'bouveret-pk', 'clarens-pk', 'crans-pk', 'cully-pk',
       'gland-pk', 'grangettes-pk', 'hermance-pk', 'lutry-pk', 'pichette-pk',
       'port-choiseul-pk', 'rolle-pk', 'savoniere-pk', 'versoix-pk',
       'preverenges-pk', 'tolochenaz-pk', 'vidy-pk']


prb = land_use[land_use.slug.isin(two)]

# some of the 2022 data has the same location name as the 2015-2021 data
# these need to be flagged. stakeholders should find away to prevent this
n_here = ["versoix-pk", "preverenges-pk", "tolochenaz-pk", "vidy-pk", "versoix-pk"]
prb_not = prb[~prb.slug.isin(n_here)].copy()
prb_not["new"] = prb_not.slug.apply(lambda x: x[:-3])
prb_not["slug"] = prb_not.new
prb_not = prb_not.drop_duplicates(["slug", "magnitude", "use"])

prb_with = prb[prb.slug.isin(n_here)].copy()
prb_with["new"] = prb_with.slug.apply(lambda x: x[:-1])
prb_with["slug"] = prb_with.new
prb_with = prb_with.drop_duplicates(["slug", "magnitude", "use"])
prb_not.drop("new", axis=1, inplace=True)
prb_with.drop("new", axis=1, inplace=True)

not_in_here = land_use[~land_use.slug.isin(two)]

land_use=pd.concat([not_in_here, prb_not, prb_with])

def apply_land_use(x, amap):
    try:
        val = amap.loc[x, "scaled"]
    except:
        val = 0
    return val
lu_map = land_use[land_use.use == "Siedl"][["slug", "scaled"]].set_index("slug")
land_use.to_csv("resources/end_pipe/land_use_work_data.csv", index=False)

### Cantonal

La situation géographique ciblée. Cependant, les échantillons du Rhône et du Léman partagent des origines communes. Pour le Léman les échantillons du Valais représentent 5% de tous les échantillons et 12% en quantité.

```{note}
Cela devrait affecter les décisions d’échantillonnage au niveau régional. Pour le Valais le bilan n'est pas correct compte tenu des kilomètres du Rhône.

Il existe cependant un consensus entre 2021 et 2022 en référence aux valeurs élevées à l'embouchure du Rhône.
```

In [8]:
kntn = pd.read_csv("resources/kantons_2021.csv")
kanton_columns =  [    
    'NAME',
    'water_name',
    'water']
kntn.set_index("slug", inplace=True, drop=True)
not_in_here = ["vidy-pk", "versoix-pk", "tolochenaz-pk", "preverenges-pk"]
k = pd.read_csv("resources/kantons_2022.csv")
k = k.dropna()
kv = k[~k.slug.isin(not_in_here)].copy()
knv = k[k.slug.isin(not_in_here)].copy()
knv["loc"] = knv.slug.apply(lambda x: x[:-1])
kv["loc"] = kv.slug.apply(lambda x: x[:-3])
new_k = pd.concat([kv, knv])
new_k.set_index("loc", inplace=True, drop=True)
kantons =  pd.concat([new_k[kanton_columns], kntn[kanton_columns]])
kantons.loc[["lavey-les-bains", "lavey-les-bains-2"], "NAME"] = "Vaud"
kt_map = kantons.NAME
combined["canton"] = combined.location.apply(lambda x: kt_map.loc[x])
combined = combined.dropna()
combined.drop_duplicates(["loc_date", "code"], inplace=True)

In [9]:
operation = {"quantity":"sum", "city":"nunique", "loc_date":"nunique", "location":"nunique"}

display_names_fr.update({
    "q_weight": "% de totale",
    "s_weight": "% des échantillons",
})
    

totals = combined.agg(operation)
canton = combined.groupby("canton").agg(operation)

canton["q_weight"] = (canton.quantity/totals.quantity).round(3)
canton["s_weight"] = (canton.loc_date/totals.loc_date).round(3)

canton.loc["Total"] = canton.sum()
canton[list(operation.keys())] = canton[operation.keys()].astype("int")

canton.index.name = None
caption = "Nous comptons 98 921 objets pour la période indiquée. \
De 25 villes. Il y avait 319 échantillons provenant de 56 sites."

canton.rename(columns=display_names_fr).style.set_table_styles(table_large_font).format('{:,}').set_caption(caption)

,quantité,municipalités,échantillons,site,% de totale,% des échantillons
Genève,"13,018",4,50,11,0.132,0.157
Valais,"12,193",2,17,2,0.123,0.053
Vaud,"73,710",19,252,43,0.745,0.79
Total,"98,921",25,319,56,1.0,1.0


### Régional

Ces étiquettes sont une façon de diviser le lac. Le transport et le dépôt d'objets dans ces zones peuvent avoir une importance régionale. Si tel est le cas, le Valais est un contributeur important.

In [10]:
regions = pd.read_csv("resources/lac_leman_regions.csv")
regions.drop_duplicates(inplace=True)

In [11]:
# rename plastock locations that have the same name as iqaasl
p_regions = pd.read_csv("resources/plastock_regions.csv")
p_regions.drop_duplicates(inplace=True)
pnot = p_regions[~p_regions.slug.isin(not_in_here)].copy()
pin = p_regions[p_regions.slug.isin(not_in_here)].copy()
pin["loc"] = pin.slug.apply(lambda x: x[:-1])
pnot["loc"] = pnot.slug.apply(lambda x: x[:-3])
new_p = pd.concat([pin, pnot])
new_p.drop("slug", axis=1, inplace=True)
new_p.rename(columns={"loc":"slug"} , inplace=True)

In [12]:
region_map = pd.concat([new_p[["slug", "alabel"]], regions[["slug", "alabel"]]])
region_map.drop_duplicates(["slug", "alabel"], inplace=True)
rg_map = region_map.set_index("slug")

com_locs = combined.location.unique()
combined["region"] = combined.location.apply(lambda x: rg_map.loc[x, "alabel"])

In [13]:
regional = combined.groupby("region").agg(operation)
regional["q_weight"] = (regional.quantity/totals.quantity).round(3)
regional["s_weight"] = (regional.loc_date/totals.loc_date).round(3)
regional.loc["Total"] = regional.sum()
regional[list(operation.keys())] = regional[operation.keys()].astype("int")
regional.index.name = None
caption_regional_fr = "Les noms régionaux représentent différentes caractéristiques \
hydrologiques ainsi que des différences géographiques."
regional.rename(columns=display_names_fr).style.format(precision=3).set_table_styles(table_large_font).format('{:,}').set_caption(caption_regional_fr)

,quantité,municipalités,échantillons,site,% de totale,% des échantillons
Grand lac,"26,318",11,87,22,0.266,0.273
Haut lac,"58,080",9,157,20,0.587,0.492
Petit lac,"14,523",6,75,14,0.147,0.235
Total,"98,921",26,319,56,1.0,1.0


### Municipal

L'unité de base. Les communes non inscrites sur la liste n'ont pas été échantillonnées.

In [14]:
municipal = combined.groupby("city").agg({"quantity":"sum", "loc_date":"nunique", "location":"nunique"})

In [15]:
municipal["q_weight"] = (municipal.quantity/totals.quantity).round(3)
municipal["s_weight"] = (municipal.loc_date/totals.loc_date).round(3)
municipal.loc["Total"] = municipal.sum().round(2)
municipal[["quantity", "loc_date", "location"]] = municipal[["quantity", "loc_date", "location"]].astype("int")
municipal.index.name = None
caption_municipal_fr = "Montreux et Vevey représentent 31% des échantillons depuis 2015."
municipal.rename(columns=display_names_fr).style.set_table_styles(table_large_font).format('{:,}').set_caption(caption_municipal_fr)

,quantité,échantillons,site,% de totale,% des échantillons
Allaman,631,3,1,0.006,0.009
Aubonne,193,4,1,0.002,0.013
Bourg-en-Lavaux,121,2,1,0.001,0.006
Bouveret,"3,286",4,1,0.033,0.013
Clarens,"1,128",4,1,0.011,0.013
Crans,21,3,1,0.0,0.009
Cully,48,4,1,0.0,0.013
Genève,"8,666",33,6,0.088,0.103
Gland,"1,690",26,3,0.017,0.082
Grangettes,"3,677",4,1,0.037,0.013


### Échantillons de bord des rivières


Les rivières représentent un environnement d'échantillonnage différent de celui des lacs. La marque des hautes eaux change plus fréquemment et avec plus d'ampleur sur les rivières. Ces changements sont si importants que les relevés effectués sur les berges impliquent souvent de retirer les sacs des arbres.

```{attention}

Les acteurs régionaux doivent se mettre d'accord sur les lieux d'échantillonnage optimaux le long de la voie navigable. Cela implique l'apport d'hydrologues et d'autres disciplines qui pourraient indiquer des lieux d'accumulation.

Il s'agit notamment d'examiner attentivement les limites de la zone d'enquête et la manière de définir ces limites sur le terrain.
``` 

In [16]:
v_cols = [
    'loc_date',
    'location', 
    'city',
    'date', 
    'doy', 
    'Project', 
    'code',
    'pcs/m', 
    'quantity', 
    'land use'
]
rt_cols = [
    'loc_date', 
    'location',
    'date',
    'water_name_slug',
    'river_bassin',
    'city',
    'length',
    'code',   
    'doy',
    'land use',
    'Project',
    'zone',
    'quantity',
    'pcs/m']

# rivers
# rt_data = pd.read_csv("resources/u_tt_data.csv")
# rt_data = rt_data[rt_data.location != 'monte-generoso'].copy()
# rt_data["date"] = pd.to_datetime(rt_data["date"], format="%Y-%m-%d")
# rt_data["doy"] = rt_data["date"].dt.dayofyear
# rt_data["date"] = rt_data["date"].dt.date
# rt_data["zone"] = "rivière"

# rt_data["land use"] = rt_data.location.apply(lambda x: apply_land_use(x, lu_map))
# rt_data["Project"] = "training"
# rt_data.rename(columns={"pcs_m":"pcs/m"}, inplace=True)

# Gcaps = [ "G21", "G23", "G24" ]

# with_caps = rt_data[rt_data.code.isin(Gcaps)].copy()
# no_caps= rt_data[~rt_data.code.isin(Gcaps)].copy()

# with_caps["code"] = "Gcaps"
# with_caps = with_caps.groupby(rt_cols[:-2], as_index=False).agg({"pcs/m":"sum", "quantity":"sum"})

# new_rt_data = pd.concat([with_caps, no_caps])

# gfoam = ["G81", "G82", "G83"]
# plasticpcs = [ "G78", "G79", "G80", "G75", "G76", "G77"]

# with_frags = new_rt_data[new_rt_data.isin(plasticpcs)].copy()
# with_foam = new_rt_data[new_rt_data.isin(gfoam)].copy()

# with_frags["code"] = "Gfrags"
# with_foam["code"] = "Gfoam"

# no_frags_foam = new_rt_data[(~new_rt_data.code.isin(plasticpcs))&(~new_rt_data.code.isin(gfoam))].copy()

# rt_gfrag = pd.concat([no_frags_foam, with_frags, with_foam])
# rt_gfrag["Project"] = "training"
# rt_gfrag["canton"] = rt_gfrag.location.apply(lambda x: kt_map.loc[x])


display_columns = [
    'loc_date', 'location', 'city', 'date', 'doy', 'Project', 'code', 'pcs/m', 'quantity', 'canton',
]

# rt_gfrag.to_csv("resources/end_pipe/rivers_work_data.csv", index=False)
rt_gfrag = pd.read_csv("resources/end_pipe/rivers_work_data.csv", low_memory=False)
rt_gfrag.drop_duplicates(["loc_date", "code"], inplace=True)
rt_gfrag.to_csv("resources/end_pipe/rivers_work_data.csv", index=False)

In [17]:
caption_rivers_fr = "Les échantillons de rivière ont la même structure que \
les échantillons de lac. La plupart des échantillons de rivière ont été prélevés en 2018."
rt_gfrag.set_index("loc_date", inplace=True, drop=True)
rt_gfrag.index.name=None


rt_gfrag[display_columns[1:]].rename(columns=display_names_fr).head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3).set_caption(caption_rivers_fr)

,site,municipalités,date,jdl,Projet,code,pcs/m,quantité,canton
"('aare-limmatspitz', '2020-07-13')",aare-limmatspitz,Gebenstorf,2020-07-13,195.000,training,Gcaps,0.000,0.000,Aargau
"('aare-port', '2021-04-23')",aare-port,Port,2021-04-23,113.000,training,Gcaps,0.050,2.000,Bern
"('aare-solothurn-lido-strand', '2020-09-05')",aare-solothurn-lido-strand,Solothurn,2020-09-05,249.000,training,Gcaps,0.000,0.000,Solothurn
"('aare_bern_caveltin', '2017-04-02')",aare_bern_caveltin,Muri bei Bern,2017-04-02,92.000,training,Gcaps,0.000,0.000,Bern
"('aare_bern_caveltin', '2017-05-06')",aare_bern_caveltin,Muri bei Bern,2017-05-06,126.000,training,Gcaps,0.000,0.000,Bern


### Échantillons sentiers pédèstre

Le lien entre les résultats de ces sites et le lac n'est pas clair. Nous ne disposons pas de données antérieures à 2020 pour les Alpes et les associations n'ont pas manifesté d'intérêt soutenu pour continuer ce projet.

```{important}
Ces résultats sont intéressants car ils représentent des zones où l'on ne s'attend pas à trouver des déchets. Dans ce contexte, les échantillons prélevés sur les sentiers de randonnée sont équivalents à ceux prélevés sur les plages. Des efforts considérables sont également déployés à tous les niveaux pour entretenir ces ressources.

L'élaboration d'un calendrier d'échantillonnage et la définition d'objectifs régionaux à cet égard peuvent inciter d'autres acteurs à agir.
```

In [18]:
sp_d = pd.read_csv("resources/checked_alpes_survey_data.csv")

sp_d["date"] = pd.to_datetime(sp_d["date"], format="%Y-%m-%d")
sp_d["doy"] = sp_d["date"].dt.dayofyear
sp_d["date"] = sp_d["date"].dt.date
sp_d.rename(columns={"pcs_m":"pcs/m", "% to buildings":"land use"}, inplace=True)
sp_d["zone"] = "montagne"
sp_d["Project"] = "testing"

Gcaps = [ "G21", "G23", "G24" ]

with_caps = sp_d[rt_cols][sp_d.code.isin(Gcaps)].copy()
no_caps= sp_d[rt_cols][~sp_d.code.isin(Gcaps)].copy()

with_caps["code"] = "Gcaps"
with_caps = with_caps.groupby(rt_cols[:-2], as_index=False).agg({"pcs/m":"sum", "quantity":"sum"})

sp_gfrag = pd.concat([with_caps, no_caps])
alpes_locs =  {
    'airolo': "Ticino" ,
    'andermatt': "Uri" ,
    'cabanes-des-diablerets': "Vaud",
    'charmey': "Valais",
    'crans-montana': "Valais",
    'grindelwald': "Bern",
    'la-berra': "Fribourg",
    'la-robella':"Neuchâtel",
    'la-tzoumaz':  "Valais",
    'les-crosets': "Valais",
    'les-diablerets': "Vaud",
    'les-paccots': "Fribourg",
    'morgins':"Valais",
    'nendaz': "Valais",
    'san-bernardino': "Grisons" ,
    'val-calanca': "Ticino",
    'verbier':"Valais",
    'veysonnaz': "Valais",
    'villars': "Vaud",
    'monte-generoso': "Ticino"
}
sp_gfrag["canton"] = sp_gfrag.location.apply(lambda x: alpes_locs[x])
sp_gfrag["zone"] = "montagne"
sp_gfrag = sp_gfrag[sp_gfrag.location != 'veysonnaz'].copy()
sp_gfrag.loc[sp_gfrag.location == "crans-montana", "city"] = "Crans-Montana"
sp_gfrag.to_csv("resources/end_pipe/work_data_alpes.csv", index=False)
sp_gfrag.set_index("loc_date", inplace=True, drop=True)
sp_gfrag.index.name = None

In [19]:
caption_trails_fr = "Les données relatives aux sentiers de randonnée proviennent du rapport fédéral. \
Elles ont été réalisées par deux personnes au cours d'une saison estivale."

sp_gfrag[display_columns[1:]].rename(columns=display_names_fr).head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3).set_caption(caption_trails_fr)

,site,municipalités,date,jdl,Projet,code,pcs/m,quantité,canton
"('clean-up-tour-airolo', '2021-08-05')",airolo,Airolo,2021-08-05,217,testing,Gcaps,0.000,0,Ticino
"('clean-up-tour-andermatt', '2021-06-19')",andermatt,Andermatt,2021-06-19,170,testing,Gcaps,0.000,0,Uri
"('clean-up-tour-cabanes-des-diablerets', '2021-08-28')",cabanes-des-diablerets,Ormont-Dessus,2021-08-28,240,testing,Gcaps,0.000,0,Vaud
"('clean-up-tour-charmey', '2021-05-08')",charmey,Val-de-Charmey,2021-05-08,128,testing,Gcaps,0.100,3,Valais
"('clean-up-tour-crans-montana', '2021-06-12')",crans-montana,Crans-Montana,2021-06-12,163,testing,Gcaps,0.000,0,Valais


### Données dimensionnelles

Nous ne listons pas les poids ici. Les méthodes utilisées pour enregistrer les poids diffèrent d'un groupe à l'autre. Cependant nous disposons de la longueur du rivage ou du sentier, de la superficie et du temps en minutes.

```{note}
Le temps nécessaire à la réalisation d'un inventaire est souvent sous-estimé. Des inventaires de plusieurs milliers d'unités peuvent nécessiter 8 à 12 heures de comptage, mais seulement une heure de collecte.

En fin de compte, la précision de vos données dépend des ressources allouées à cet effet. D'après notre expérience, les gens participent à la collecte mais pas au comptage.
```

In [20]:
iq_d = pd.read_csv("resources/corrected_dims.csv")
alp_d = pd.read_csv("resources/alpes_dims.csv")
ps_d = pd.read_csv("resources/asl_beaches2.csv")


ps_d["slug"] = ps_d.location.apply(lambda x: slugify.slugify(x))
ps_d["loc"] = ps_d.slug.apply(lambda x: f'{x}-pk')

ps_da = ps_d.groupby("location").agg({"area":"sum", "length":"sum", "time":"sum"})

two = ['aubonne-pk', 'baby-plage-pk', 'bouveret-pk', 'clarens-pk', 'crans-pk', 'cully-pk',
       'gland-pk', 'grangettes-pk', 'hermance-pk', 'lutry-pk', 'pichette-pk',
       'port-choiseul-pk', 'rolle-pk', 'savoniere-pk', 'versoix-pk',
       'preverenges-pk', 'tolochenaz-pk', 'vidy-pk']

n_here = ["versoix-pk", "preverenges-pk", "tolochenaz-pk", "vidy-pk", "versoix-pk"]
prb_not = ps_d[~ps_d["loc"].isin(n_here)].copy()
prb_not["new"] = prb_not["loc"].apply(lambda x: x[:-3])
prb_not["slug"] = prb_not.new

prb_with = ps_d[ps_d["loc"].isin(n_here)].copy()
prb_with["new"] = prb_with["loc"].apply(lambda x: x[:-1])
prb_with["slug"] = prb_with.new
pk_dims = pd.concat([prb_with, prb_not])
pk_dims["location"] = pk_dims["new"]
pk_dim = pk_dims.groupby("location").agg({"area":"sum", "length":"sum", "time":"sum"})

alp_dims = alp_d[["survey_key", "length", "area", "time_minutes"]].copy()
alp_dims["loc"] = alp_dims.survey_key.apply(lambda x: x[14:])
alp_dims.rename(columns={"time_minutes":"time"}, inplace=True)
alpes_locs = sp_d.location.unique()
def i_f_this_name_is_here(x, alist):
    for name in alist:
        if name in x:
            return name
alp_dims["location"] = alp_dims["loc"].apply(lambda x: i_f_this_name_is_here(x, alpes_locs))
alp_dim = alp_dims.groupby("location").agg({"area":"sum", "length":"sum", "time":"sum"})

iq_dims = iq_d[["survey_key", "date", "length", "area", "time_minutes"]].copy()

iq_dims["suff"] = list(zip(iq_dims["date"], iq_dims["length"]))
iq_dims["suff"] = iq_dims.suff.apply(lambda x: "".join((str(x[0]), str(x[1]))))
iq_dims.rename(columns={"time_minutes":"time"}, inplace=True)

iqdx = iq_dims[["survey_key", "suff"]].copy()
these = iqdx.suff.values
locations = [x.replace(these[i], "") for i, x in enumerate(iqdx.survey_key.values)]
iq_dims["location"] = locations
iq_dim = iq_dims.groupby("location").agg({"area":"sum", "length":"sum", "time":"sum"})

working_dimensions=pd.concat([alp_dim, iq_dim, pk_dim])
working_dimensions.to_csv('resources/work_dimensions.csv')

working_dimensions.index.name = None
display_names_fr.update({"time":"minutes"})

caption_dims_fr = "l'ampleur de l'effort en termes de surface, de distance et de temps."
working_dimensions.rename(columns=display_names_fr).head().style.set_table_styles(table_large_font).format(precision=3).set_caption(caption_dims_fr)

,m²,mètres,minutes
airolo,57.000,38,99
andermatt,800.000,100,220
cabanes-des-diablerets,18.000,12,199
charmey,150.000,30,220
crans-montana,64.000,43,149


### Test

Avant de passer à l'étape suivante. Nous vérifions les données par rapport au rapport fédéral. Nos résultats ici doivent correspondre à la page ici [Le Léman 2021](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/lac-leman.html)

In [21]:
# check the 2021 results against the pubished federal report

start_date, end_date = pd.to_datetime("2020-04-01"), "2021-05-31" 
combined["date"] = pd.to_datetime(combined["date"])
mask = (combined["date"] >= start_date) & (combined["date"] <= end_date)
iqaasl = combined[mask]
codes = ["Gfrags", "G27", "G67", "G95", "G112", "G30", "G70", "G89"]

t_c_v = iqaasl[iqaasl.code.isin(codes)]
tcv = t_c_v.groupby(["loc_date", "code"], as_index=False).agg({"pcs/m":"sum", "quantity":"sum"})
tfg = tcv.groupby("code", as_index=False)["pcs/m"].median().T
tfg.columns = tfg.loc["code"].values
tfg.drop("code", inplace=True)
caption_test_fr = "Ces valeurs doivent correspondre à la médiane déclarée pour le même objet dans le rapport fédéral. "
tfg.style.set_table_styles(table_large_font).format(precision=3).set_caption(caption_test_fr)

,G112,G27,G30,G67,G70,G89,G95,Gfrags
pcs/m,0.000,0.470,0.215,0.090,0.010,0.055,0.120,0.615


## L'ensemble de données pour le lac

Les résultats de l'enquête contiennent désormais toutes les informations décrites ici ou permettent d'accéder à d'autres informations sur l'événement.

In [22]:
combined["land use"] = combined.location.apply(lambda x: apply_land_use(x, lu_map))
combined["zone"] = "lac"
combined.to_csv("resources/end_pipe/survey_work_data.csv", index=False)
combined["date"] = combined["date"].dt.date

caption_last_fr = "Résultats pour les emballages de bonbons. Les données de l'enquête comportent \
des étiquettes pour les relations inter et intra cantonales. \
C'est essentiel pour les replacer dans leur contexte."

combined[combined.code == "G30"].rename(columns=display_names_fr).head().style.set_table_styles(table_css_styles).format(precision=3).set_caption(caption_last_fr)

,échantillons,site,water_name_slug,municipalités,date,jdl,Projet,code,pcs/m,quantité,canton,region,land use,zone
117,"('anarchy-beach', '2018-04-02')",anarchy-beach,lac-leman,La Tour-de-Peilz,2018-04-02,92,training,G30,0.500,35,Vaud,Haut lac,1.000,lac
293,"('anarchy-beach', '2021-03-24')",anarchy-beach,lac-leman,La Tour-de-Peilz,2021-03-24,83,training,G30,0.210,6,Vaud,Haut lac,1.000,lac
469,"('arabie', '2016-01-24')",arabie,lac-leman,Vevey,2016-01-24,24,training,G30,10.320,413,Vaud,Haut lac,0.960,lac
645,"('arabie', '2016-04-02')",arabie,lac-leman,Vevey,2016-04-02,93,training,G30,0.520,21,Vaud,Haut lac,0.960,lac
821,"('arabie', '2016-08-25')",arabie,lac-leman,Vevey,2016-08-25,238,training,G30,0.650,26,Vaud,Haut lac,0.960,lac


In [23]:

today = dt.datetime.now().date().strftime("%d/%m/%Y")
where = "Biel, CH"

my_block = f"""
<br />
<br />
This script updated {today} in {where}

\u2764\ufe0f what you do - faites ce que vous \u2764\ufe0f
"""

md(my_block)


<br />
<br />
This script updated 04/09/2023 in Biel, CH

❤️ what you do - faites ce que vous ❤️


In [24]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/cantonal_reports.git

Git branch: main

slugify: 8.0.1
numpy  : 1.25.2
pandas : 2.0.3



_analyst at hammerdirt_